In [1]:
import sys
sys.path.append("../../code")

In [2]:
import gen

In [3]:
import torch
import torch.nn as nn

In [4]:
class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.fc_layer = nn.Sequential(
            nn.Linear(28*28,1024),
            nn.ReLU(inplace=True),
            
            nn.Linear(1024, 4096),
            nn.ReLU(inplace=True),
            
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            
            nn.Linear(1024, 10),
            nn.Sigmoid()
    )
    def forward(self, x):
        x = x.view(x.size(0), 28*28)
        x = self.fc_layer(x)
        return x

In [5]:
model = FNN()

In [6]:
pretrained_model="../../target/mnist/FNN.pt"
model.load_state_dict(torch.load(pretrained_model))

<All keys matched successfully>

In [7]:
method = 'DEEPFOOL'
config = 'DEEPFOOL_mnist.json'
target_model = model
loss_func = None
data_set = 'mnist'
METHOD_dataset_NET = 'DEEPFOOL_mnist_FNN'
choose = 'test'

In [8]:
gen.trod_attck(
    method,
    config,
    target_model,
    loss_func,
    data_set,
    METHOD_dataset_NET,
    choose
    )

cuda
-test set start-
count:10000/10000,cost:0.030901s
test set cost:298.29553008s
saving...
-complete-


In [9]:
import tst

In [10]:
class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.fc_layer = nn.Sequential(
            nn.Linear(28*28,1024),
            nn.ReLU(inplace=True),
            
            nn.Linear(1024, 4096),
            nn.ReLU(inplace=True),
            
            nn.Linear(4096, 1024),
            nn.ReLU(inplace=True),
            
            nn.Linear(1024, 10),
            nn.Sigmoid()
    )
    def forward(self, x):
        x = x.view(x.size(0), 28*28)
        x = self.fc_layer(x)
        return x

In [11]:
t_model = FNN()

In [12]:
test_model="../../target/mnist/FNN.pt"
t_model.load_state_dict(torch.load(test_model))

<All keys matched successfully>

In [13]:
# robustness_test(model, adv_sample_path, sample_num, sample_shape)
tst.robustness_test(
    model = t_model,
    adv_sample_path = '../sample/DEEPFOOL_mnist_FNN/test',
    sample_num = 10000,
    sample_shape = [1,28,28]
)

fooling_rate:96.780000%


In [14]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2),
            nn.Conv2d(6, 16, 5),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(256, 120),
            nn.ReLU(inplace=True),
            nn.Linear(120, 84),
            nn.ReLU(inplace=True),
            nn.Linear(84, 10)
        )

    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [15]:
t_model = LeNet()

In [16]:
test_model="../../target/mnist/LeNet.pt"
t_model.load_state_dict(torch.load(test_model))

<All keys matched successfully>

In [17]:
# robustness_test(model, adv_sample_path, sample_num, sample_shape)
tst.robustness_test(
    model = t_model,
    adv_sample_path = '../sample/DEEPFOOL_mnist_FNN/test',
    sample_num = 10000,
    sample_shape = [1,28,28]
)

fooling_rate:40.280000%


In [18]:
import torch.nn.functional as F
class ResidualBlock(nn.Module):
    def __init__(self, inchannel, outchannel, stride=1):
        super(ResidualBlock, self).__init__()
        self.left = nn.Sequential(
            nn.Conv2d(inchannel,outchannel,kernel_size=3,padding=1,stride=stride,bias=False),
            nn.BatchNorm2d(outchannel),
            nn.ReLU(),
            nn.Conv2d(outchannel, outchannel, kernel_size=3, padding=1, stride=stride, bias=False),
            nn.BatchNorm2d(outchannel)
        )
        self.right = nn.Sequential()
        
        if(inchannel != outchannel):
            self.right = nn.Sequential(
                nn.Conv2d(inchannel, outchannel, kernel_size=3, padding=1, stride=stride, bias=False),
                nn.BatchNorm2d(outchannel),
            )

    def forward(self, x):
        out = self.left(x)
        out += self.right(x)
        out =F.relu(out)
        return out

class ResNet(nn.Module):
    def __init__(self, ResidualBlock, num_classes=10):
        super(ResNet, self).__init__()
        self.inchannel = 64
        self.conv1 = nn.Sequential(
            nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1,bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        )
        self.layer1 = self.make_layer(ResidualBlock, 64,  2, stride=1)
        self.layer2 = self.make_layer(ResidualBlock, 128, 2, stride=1)
        self.conv2 = nn.Conv2d(128,128,3,stride=2)
        self.layer3 = self.make_layer(ResidualBlock, 256, 2, stride=1)
        self.conv3 = nn.Conv2d(256, 256, 3, stride=2)
        #self.layer4 = self.make_layer(ResidualBlock, 512, 2, stride=1)
        self.conv4 = nn.Conv2d(256,256,6)
        self.fc = nn.Linear(256, num_classes)

    def make_layer(self, block, channels, num_blocks, stride):
        layer = []
        for i in range(num_blocks):
            layer.append(block(self.inchannel,channels,stride))
            self.inchannel = channels
        #
        return nn.Sequential(*layer)

    def forward(self, x):
        out = self.conv1(x)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.conv2(out)
        out = self.layer3(out)
        out = self.conv3(out)
        #out = self.layer4(out)
        out = self.conv4(out)
        #out = F.avg_pool2d(out,4)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

def ResNet18():
    return ResNet(ResidualBlock)


In [19]:
t_model = ResNet18()

In [20]:
test_model="../../target/mnist/ResNet18.pt"
t_model.load_state_dict(torch.load(test_model))

<All keys matched successfully>

In [21]:
tst.robustness_test(
    model = t_model,
    adv_sample_path = '../sample/DEEPFOOL_mnist_FNN/test',
    sample_num = 10000,
    sample_shape = [1,28,28]
)

fooling_rate:22.510000%
